In [3]:
import numpy as np
import pandas as pd

In [4]:
# ------------------------------------------------
# 1) Helper Function: Parsing Time-Parameterized Vectors
# ------------------------------------------------
def parse_time_vector(vector_string, num_periods):
    """
    Reads in a string vector and returns the corresponding values for each period in a numpy array.

    NOTE: Stepping to or ramping from [old rate] to [new rate] AFTER [duration periods].
    """
    tokens = vector_string.strip().split()
    
    # If there's only one token and it's just a number, interpret as constant
    if len(tokens) == 1:
        try:
            val = float(tokens[0])
            return np.full(num_periods, val)
        except ValueError:
            raise ValueError(f"Could not parse constant rate from '{vector_string}'")
    
    try:
        current_rate = float(tokens[0])
    except ValueError:
        raise ValueError(f"Could not parse the initial rate from '{tokens[0]}'")
    
    period_rates = np.full(num_periods, current_rate)
    current_period = 1  # 1-based index
    idx = 1
    
    while idx < len(tokens):
        duration_and_type = tokens[idx]
        idx += 1
        new_rate_str = tokens[idx]
        idx += 1
        
        mode = duration_and_type[-1].upper()  # 'S' or 'R'
        duration_str = duration_and_type[:-1]
        
        try:
            duration = int(duration_str)
        except ValueError:
            raise ValueError(f"Could not parse duration from '{duration_str}'")
        
        try:
            new_rate = float(new_rate_str)
        except ValueError:
            raise ValueError(f"Could not parse new rate from '{new_rate_str}'")
        
        start_period = current_period
        end_period = current_period + duration
        if end_period > num_periods:
            end_period = num_periods
        
        if mode == 'S':
            for y in range(start_period, end_period):
                if y-1 < num_periods:
                    period_rates[y-1] = current_rate
            step_period_index = end_period - 1
            if 0 <= step_period_index < num_periods:
                period_rates[step_period_index] = new_rate
            current_rate = new_rate
            current_period = end_period
        
        elif mode == 'R':
            # Ramp linearly
            rate_diff = new_rate - current_rate
            for i_period in range(duration + 1):
                frac = i_period / float(duration) if duration > 1 else 1.0
                actual_rate = current_rate + frac * rate_diff
                actual_period = start_period + i_period
                if actual_period <= num_periods:
                    period_rates[actual_period - 1] = actual_rate
            current_rate = new_rate
            current_period = end_period
        
        else:
            raise ValueError(f"Unknown mode '{mode}' in parse_time_vector.")
    
    for y in range(current_period, num_periods+1):
        if y-1 < num_periods:
            period_rates[y-1] = current_rate
    
    return period_rates


# ------------------------------------------------
# 2) Helper Function: Draw from Chosen Distribution
# ------------------------------------------------
def draw_random(dist_type, mean, stdev, size=1):
    """
    dist_type: 'normal' or 'lognormal'.
    mean, stdev refer to the *arithmetic* mean and std. dev (if lognormal).
    """
    dist_type = dist_type.lower()
    if dist_type == 'normal':
        return np.random.normal(loc=mean, scale=stdev, size=size)
    elif dist_type == 'lognormal':
        if mean <= 0:
            return np.zeros(size)
        sigma_sq = stdev**2
        mu_sq = mean**2
        phi = np.sqrt(np.log(1 + sigma_sq/mu_sq))   
        m   = np.log(mu_sq / np.sqrt(sigma_sq+mu_sq))
        return np.random.lognormal(mean=m, sigma=phi, size=size)
    else:
        raise ValueError(f"Unsupported distribution type '{dist_type}'")

In [ ]:
# ------------------------------------------------
# 3) Main Simulation Function
# ------------------------------------------------
def monte_carlo_retirement(
    num_sims,
    num_periods,
    # Inflation vectors
    inflation_rate_vector,           
    inflation_vol_vector,            
    inflation_dist='normal',
    
    # Asset returns
    asset_mean_vectors=None,
    asset_vol_vectors=None,
    asset_dist_types=None,  # e.g. ['normal','normal','normal',...], length=10
    
    # Allocations
    allocation_vectors=None,  # length=10
    rebalance=False,           # enable or disable rebal
    rebalance_frequency=1,     # how often (in years) we rebalance
    
    # Contribution parameters
    initial_contribution=10000.0,
    contrib_growth_vector="2",
    contrib_vol_vector="1",
    contrib_dist='normal',
    peg_contribution_to_inflation=True, #if true, then contribution growth value will be added to inflation value to get total contribution growth
    contribution_stop_period=30, #no contributions from this period and onward
    
    # Drawdown parameters
    first_drawdown_inflation_adjusted=True,
    initial_drawdown=0.0,
    drawdown_growth_vector="0",
    drawdown_vol_vector="0",
    drawdown_dist='normal',
    peg_drawdown_to_inflation=True, #if true, then drawdown growth value will be added to inflation value to get total drawdown growth
    drawdown_start_period=30, #drawdowns starting this period and onward      
    
    # Initial portfolio balance
    initial_balance=0.0,
    
    # For reproducibility
    random_seed=None
):
    """
    Returns a DataFrame with both nominal and real (inflation-adjusted) statistics by period.

    Order of operations for a given period:
    Period pre-growth balance = last period final balance + contribution
    Period post-growth balance = Period pre-growth balance * (1 + weighted average growth rate)
    Period final balance = Period post-growth balance - drawdown

    Contributions happen at the beginning of the period, then inflation, then investment growth, then drawdowns, then rebalancing.     
    Therefore, contributions are inflation adjusted up through the previous period's cumulative inflation. 
    Drawdowns are inflation adjusted up through the current period's cumulative inflation.
    """
    if random_seed is not None:
        np.random.seed(random_seed)

    if not rebalance:
        rebalance_frequency = 1
    
    # 1) Parse and convert all time-varying vectors to period arrays
    inflation_mean_arr = parse_time_vector(inflation_rate_vector, num_periods)
    inflation_std_arr  = parse_time_vector(inflation_vol_vector, num_periods)
    
    # parse asset return vectors
    num_asset_classes = 10
    asset_mean_arr = np.column_stack([parse_time_vector(v, num_periods) for v in asset_mean_vectors])
    asset_vol_arr  = np.column_stack([parse_time_vector(v, num_periods) for v in asset_vol_vectors])
    
    # parse model allocations
    if rebalance:
        asset_model_alloc_arr = np.column_stack([parse_time_vector(v, num_periods) for v in allocation_vectors]) 
    else:
        #model allocations stay the same throughout:
        first_row = np.array([parse_time_vector(v, num_periods)[0] for v in allocation_vectors])
        asset_model_alloc_arr = np.tile(first_row, (num_periods, 1))
        
    # parse contributions
    contrib_mean_arr = parse_time_vector(contrib_growth_vector, num_periods)
    contrib_std_arr  = parse_time_vector(contrib_vol_vector, num_periods)
    
    # parse drawdowns
    drawdown_mean_arr = parse_time_vector(drawdown_growth_vector, num_periods)
    drawdown_std_arr  = parse_time_vector(drawdown_vol_vector, num_periods)
    
    # 2) Prepare arrays for results
    all_asset_allocs = np.zeros((num_sims, num_periods, num_asset_classes))

    #nominal values:
    all_balances = np.zeros((num_sims, num_periods))        
    all_asset_balances = np.zeros((num_sims, num_periods, num_asset_classes)) 
    all_inflations = np.zeros((num_sims, num_periods))       
    all_contributions = np.zeros((num_sims, num_periods))    
    all_drawdowns = np.zeros((num_sims, num_periods))        
    
    #Real values:
    all_real_balances = np.zeros((num_sims, num_periods))
    all_real_asset_balances = np.zeros((num_sims, num_periods, num_asset_classes)) 
    all_real_contributions = np.zeros((num_sims, num_periods))
    all_real_drawdowns = np.zeros((num_sims, num_periods))
    all_cum_inflation = np.ones((num_sims, num_periods))

    # 3) Simulation loop
    for s in range(num_sims):
        cum_infl = 1.0
        for t in range(num_periods):
            # 3.1 inflation
            infl_mean_t = inflation_mean_arr[t] / 100.0
            infl_std_t = inflation_std_arr[t] / 100.0
            this_infl = draw_random(inflation_dist, infl_mean_t, infl_std_t, 1)[0]
            cum_infl *= (1.0 + this_infl)
            all_cum_inflation[s, t] = cum_infl
            
            # 3.2 contributions
            c_growth_mean_t = contrib_mean_arr[t] / 100.0
            c_growth_std_t = contrib_std_arr[t] / 100.0
            c_draw = draw_random(contrib_dist, c_growth_mean_t, c_growth_std_t, 1)[0]
            if peg_contribution_to_inflation:
                c_growth = c_draw + this_infl
            else:
                c_growth = c_draw
            
            if t == 0:
                this_contribution = initial_contribution
            elif t >= contribution_stop_period:
                this_contribution = 0.0
            else:
                prev_contrib = all_contributions[s, t - 1]
                this_contribution = prev_contrib * (1 + c_growth)
            
            # distributing contributions pro rata 
            # if rebalencing happens, the previous asset allocations are already rebalanced
            alloc_fracs = asset_model_alloc_arr[t] / 100.0
            if t == 0:
                all_asset_balances[s, t] = alloc_fracs * (this_contribution + initial_balance)
            else:
                prev_alloc_fracs = all_asset_allocs[s, t - 1] / 100.0
                total_val = all_asset_balances[s, t - 1].sum()
                if total_val < 1e-9:
                    all_asset_balances[s, t] = alloc_fracs * this_contribution
                else:
                    all_asset_balances[s, t] = all_asset_balances[s, t - 1] + prev_alloc_fracs * this_contribution
            
            # 3.3 Asset returns: 
            for i in range(10):
                mean_i = asset_mean_arr[t, i] / 100.0
                std_i = asset_vol_arr[t, i] / 100.0
                dist_i = asset_dist_types[i]
                ret_i = draw_random(dist_i, mean_i, std_i, 1)[0]
                all_asset_balances[s, t, i] *= (1.0 + ret_i)
        
            # 3.4 Drawdowns
            if t < drawdown_start_period:
                this_drawdown = 0.0
            elif t == drawdown_start_period:
                if first_drawdown_inflation_adjusted:
                    this_drawdown = initial_drawdown * cum_infl
                else:
                    this_drawdown = initial_drawdown
            else:
                d_mean_t = drawdown_mean_arr[t] / 100.0
                d_std_t = drawdown_std_arr[t] / 100.0
                d_draw = draw_random(drawdown_dist, d_mean_t, d_std_t, 1)[0]
                if peg_drawdown_to_inflation:
                    d_growth = d_draw + this_infl
                else:
                    d_growth = d_draw
                prev_dd = all_drawdowns[s, t - 1]
                this_drawdown = prev_dd * (1 + d_growth)

            # Remove the drawdown pro rata from each asset
            total_val = all_asset_balances[s, t].sum()
            curr_alloc_fracs = np.zeros(num_asset_classes)
            if total_val > 1e-9:
                curr_alloc_fracs = all_asset_balances[s, t] / total_val
            
            if this_drawdown > total_val:
                #liquidating the entire portfolio
                this_drawdown = total_val
            elif total_val < 1e-9:
                this_drawdown = 0
            else:
                all_asset_balances[s, t] -= curr_alloc_fracs * this_drawdown

            # 3.5 Rebalancing
            if rebalance and (t + 1) % rebalance_frequency == 0:
                target_alloc_fracs = asset_model_alloc_arr[t] / 100.0
                total_val = all_asset_balances[s, t].sum()
                if total_val < 1e-9:
                    all_asset_balances[s, t] = np.zeros(num_asset_classes)
                else:
                    all_asset_balances[s, t] = total_val * target_alloc_fracs

            # 3.6 Store final values
            total_val = all_asset_balances[s, t].sum() 
            if total_val < 1e-9:
                all_asset_allocs[s, t] = np.zeros(num_asset_classes)
            else:
                all_asset_allocs[s, t] = 100 * all_asset_balances[s, t] / total_val
            
            final_bal = all_asset_balances[s, t].sum()
            all_balances[s, t] = final_bal
            all_inflations[s, t] = this_infl
            all_contributions[s, t] = this_contribution
            all_drawdowns[s, t] = this_drawdown
            
            all_real_balances[s, t] = final_bal / cum_infl
            all_real_asset_balances[s, t] =  all_asset_balances[s, t] / cum_infl
            all_real_contributions[s, t] = this_contribution / (cum_infl /(1 + this_infl))
            all_real_drawdowns[s, t] = this_drawdown / cum_infl
    
    # 4) Summaries across sims
    percentiles = [10, 25, 50, 75, 90]
    periods = range(1, num_periods + 1)
    output_dict = {'period': list(periods)}
    
    for p in percentiles:
        output_dict[f"Nominal_Balance_P{p}"] = [np.percentile(all_balances[:, t], p) for t in range(num_periods)]
    for p in percentiles:
        output_dict[f"Real_Balance_P{p}"] = [np.percentile(all_real_balances[:, t], p) for t in range(num_periods)]
    
    # output_dict['Avg_Effective_Return'] = [
    #     #NEED TO FIX
    # ]
    output_dict['Median_Inflation'] = [
        np.median(all_inflations[:, t])*100.0 for t in range(num_periods)
    ]
    output_dict['Median_Nominal_Contribution'] = [
        np.median(all_contributions[:, t]) for t in range(num_periods)
    ]
    output_dict['Median_Real_Contribution'] = [
        np.median(all_real_contributions[:, t]) for t in range(num_periods)
    ]
    output_dict['Median_Nominal_Drawdown'] = [
        np.median(all_drawdowns[:, t]) for t in range(num_periods)
    ]
    output_dict['Median_Real_Drawdown'] = [
        np.median(all_real_drawdowns[:, t]) for t in range(num_periods)
    ]
    for i in range(10):
        output_dict[f"Median_Alloc_Asset{i+1}"] = np.median(all_asset_allocs[:, :, i], axis=0)  

    df_results = pd.DataFrame(output_dict)
    return df_results

In [16]:
num_sims = 1000
num_periods = 80

asset_mean_vecs = [
    "8", 
    "6", 
    "5", 
    "7", 
    "9", 
    "4", 
    "3", 
    "5 10S 7", 
    "10", 
    "2",
]
asset_vol_vecs = [
    "15", 
    "12", 
    "10", 
    "13", 
    "18", 
    "8", 
    "5", 
    "10 10S 12", 
    "20", 
    "3",
]
asset_dist_types = ["normal"] * 10  

allocation_vecs = [
    "70",   
    "30",   
    "0",   
    "0",   
    "0",   
    "0",    
    "0",    
    "0",    
    "0",    
    "0",    
]

df_results = monte_carlo_retirement(
    num_sims = num_sims,
    num_periods = num_periods,
    inflation_rate_vector = "3",   
    inflation_vol_vector = "0",                
    inflation_dist = 'normal',
    
    asset_mean_vectors = asset_mean_vecs,
    asset_vol_vectors = asset_vol_vecs,
    asset_dist_types = asset_dist_types,
    allocation_vectors = allocation_vecs,
    rebalance = False,
    rebalance_frequency = 1,
    
    initial_contribution = 10000.0,
    contrib_growth_vector = "0",   
    contrib_vol_vector = "0",          
    contrib_dist = 'normal',
    peg_contribution_to_inflation = True,
    contribution_stop_period = 50, 
    
    first_drawdown_inflation_adjusted = True,
    initial_drawdown = 800000.0, 
    drawdown_growth_vector = "0",  
    drawdown_vol_vector = "0",
    drawdown_dist = 'normal',
    peg_drawdown_to_inflation = True,
    drawdown_start_period = 50, 
    
    initial_balance = 0.0,
    random_seed = 42
)
df_results.to_excel("output.xlsx")
df_results  


,period,Nominal_Balance_P10,Nominal_Balance_P25,Nominal_Balance_P50,Nominal_Balance_P75,Nominal_Balance_P90,Real_Balance_P10,Real_Balance_P25,Real_Balance_P50,Real_Balance_P75,...,Median_Alloc_Asset1,Median_Alloc_Asset2,Median_Alloc_Asset3,Median_Alloc_Asset4,Median_Alloc_Asset5,Median_Alloc_Asset6,Median_Alloc_Asset7,Median_Alloc_Asset8,Median_Alloc_Asset9,Median_Alloc_Asset10
0,1,9.340695e+03,1.002562e+04,10769.526676,11508.917358,12139.793339,9.068636e+03,9.733616e+03,1.045585e+04,11173.706173,...,70.474231,29.525769,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,1.919170e+04,2.080955e+04,22518.675677,24323.639140,26185.201696,1.809002e+04,1.961499e+04,2.122601e+04,22927.362749,...,70.297332,29.702668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,2.965077e+04,3.231631e+04,35201.402098,38727.419662,42012.802656,2.713465e+04,2.957400e+04,3.221427e+04,35441.075092,...,70.908473,29.091527,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,4.051696e+04,4.465817e+04,49633.375499,54531.062314,59333.432875,3.599880e+04,3.967820e+04,4.409861e+04,48450.142575,...,71.368002,28.631998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,5.203339e+04,5.768695e+04,64900.055838,72895.021706,79356.641425,4.488446e+04,4.976127e+04,5.598336e+04,62879.886061,...,71.114411,28.885589,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,76,2.460692e-07,8.831524e-06,0.000193,0.004719,0.081967,2.602724e-08,9.341284e-07,2.045786e-05,0.000499,...,87.745681,11.708976,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
76,77,8.464704e-08,2.638792e-06,0.000073,0.002209,0.027547,8.692516e-09,2.709810e-07,7.456565e-06,0.000227,...,88.042098,11.226867,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
77,78,2.460055e-08,9.499103e-07,0.000028,0.000845,0.015511,2.452682e-09,9.470634e-08,2.786560e-06,0.000084,...,88.195475,10.255628,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
78,79,1.055727e-08,3.341636e-07,0.000010,0.000405,0.005824,1.021906e-09,3.234583e-08,9.960555e-07,0.000039,...,88.435059,9.454190,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
